In [1]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
from helper_functions import load_dataset, save_dataset

### Loading the dataset

In [2]:
data = load_dataset("../data/assignment1_income_levels.xlsx")

## 1) Data Inspection and Preparation

First, we do some basic inspection of the dataset, such as checking the first few rows, the data types, the amount of missing values, and the value counts of certain columns.

In [3]:
data.head()

In [4]:
data.info()

In [5]:
data.describe()

In [6]:
data.isnull().sum() # check amount of missing values per column

In [7]:
# percentage of missing values in 'ability to speak english' column
data['ability to speak english'].isnull().sum() / len(data) * 100

In [8]:
# percentage of missing values in 'gave birth this year' column
data['gave birth this year'].isnull().sum() / len(data) * 100

In [9]:
# percentage of "Yes" and "No" values in the 'gave birth this year' column
data['gave birth this year'].value_counts() / len(data) * 100

In [10]:
# percentage of "Male" and "Female" values in the 'sex' column
data['sex'].value_counts() / len(data) * 100

In [11]:
# percentages of value counts for the 'workclass' column
data['workclass'].value_counts() / len(data) * 100

In [12]:
# value counts for the 'marital status' column
data['marital status'].value_counts()

In [13]:
# check for people younger than 25, 30, 40, how many of them are 'Never married'.
never_married = [data[(data['age'] < 25) & (data['marital status'] == 'Never married')].shape[0],
                 data[(data['age'] < 30) & (data['marital status'] == 'Never married')].shape[0],
                 data[(data['age'] < 40) & (data['marital status'] == 'Never married')].shape[0]]
# get total count of people who are never married
total_never_married = data[data['marital status'] == 'Never married'].shape[0]
# get the percentage of people who are never married
never_married_percentages = [x / total_never_married * 100 for x in never_married]
df = pd.DataFrame({'Age Group': ['<25', '<30', '<40'], 'Count of Never Married': never_married, 'Percentage of Never Married': never_married_percentages})

In [14]:
df

We can see that there are way more men than women (66.7% vs 33.3%) in the dataset. People are mostly working in the 'private' sector (73.8%), with the second being 'governmental' (15.7%). The husband to wife ratio is a bit more than 2:1, which is reasonable considering the 'sex' column distribution. If we look at people younger than 40, we are already at 82% of the total amount of people who are 'Never married'. It seems to be decently correlated to age, which makes sense of course.

In [15]:
# check the amount of missing values in the 'gave birth this year' column for the age bins (17-28, 28-38, 38-49, 49-65, 65-93) for females
female_data = data[data['sex'] == 'Female']
female_data['gave birth this year'][female_data['gave birth this year'].isnull()].groupby(pd.cut(female_data['age'], bins=[0,28,38,49,65,93]), observed=False).size()

We can see that the missing values in the 'gave birth this year' column are only present in the 49-93 age range. Since it was proven to be a higher health risk to give birth after ~35 years old for women, this could be seen as a reason for why the missing values are mostly in the 38+ age bin (also, we have to keep in mind menopause which occurs roughly between the ages of 45 and 55 years). This will be very useful for the imputation of the missing values in the 'gave birth this year' column.
Sources used:
- https://www.acog.org/womens-health/faqs/having-a-baby-after-age-35-how-aging-affects-fertility-and-pregnancy#:~:text=The%20risks%20of%20miscarriage%20and,chance%20of%20a%20multiple%20pregnancy.
- https://www.who.int/news-room/fact-sheets/detail/menopause#:~:text=Most%20women%20experience%20menopause%20between,changes%20in%20the%20menstrual%20cycle.

### Imputing missing values & changing data types

We have 2 columns with missing values: 'ability to speak english' and 'gave birth this year'. We will impute the missing values in the 'ability to speak english' column with 0 (for native speakers), and the missing values in the 'gave birth this year' column with a 'No'. We will also change the data type of certain columns if they can be represented with smaller data types to save memory.

In [16]:
# Impute missing values in 'ability to speak English' column with 0 (for native speakers)
data['ability to speak english'] = data['ability to speak english'].fillna(0)

In [17]:
# Impute missing values in 'gave birth this year' column with a 'No'
data['gave birth this year'] = data['gave birth this year'].fillna('No')

In [18]:
# change the data type of certain columns if they can be represented with smaller data types to save memory
data['age'] = data['age'].astype('int8') # in our dataset: min age is 17 and max age is 93
data['education'] = data['education'].astype('int8') # in our dataset: min education is 1 and max education is 24
data['workinghours'] = data['workinghours'].astype('int8') # in our dataset: min working hours is 1 and max working hours is 99
data['ability to speak english'] = data['ability to speak english'].astype('int8') # in our dataset: min number of 'ability to speak english' is 0 and max ability to speak english is 4

### Binning columns: 'age', 'workinghours', 'education'

#### 'age' column: extra analysis and binning

In [19]:
# check how much a person has workinghours on average per age, for people older than 65
data['workinghours'][data['age'] > 65].groupby(data['age']).agg(['mean', 'count'])

Although pension age is around 65, the mean of most ages > 65 is still quite high (we of course keep the per-age count in mind).

In [20]:
p = sns.scatterplot(data=data['workinghours'][data['age'] > 16].groupby(data['age']).agg(['mean', 'count']).reset_index(), x='age', y='mean', size='count')
p.set(xlabel='Age', ylabel='Average workinghours', title='Average workinghours per age')
plt.show()

Above is a plot of the previously generated dataframe, with x-axis as age, y-axis as average workinghours, and the size of the point as the amount of people of that age. We can see a reverse U-shape (parabola), which is quite interesting. We can observe that people on average work less when they are younger, then work more when they are in their 30s to their 60s, and then work less again when they are older.

In [21]:
# plot of the distribution of the 'age' column for first easy inspection
p = sns.displot(data['age'], kde=True, bins=20)
p.set(xlabel='Age', ylabel='Count', title='Distribution of ages')
plt.show()

We can see that the distribution of the ages has a bit of a negative skew (increasingly lower counts for 65+), with two small peaks in roughly the ranges 25-35 and 45-65.

In [22]:
pd.qcut(data['age'], q=5, retbins=True) # quantile-based binning

In [23]:
# we will do customized binning for the 'age' column; '17-28', '28-38', '38-49', '49-65', '65-93', but this is based on the above quantile-based binning output.
# Furthermore, other reasons for this binning include the results of the average workinghours scatterplot (where we see that people in the range 17-28 and in the range 65-93 are significantly different from the other age ranges, meaning the large 65-93 group is a viable option) and the fact that the first four age ranges have a difference of roughly 10 years, which is a not a bad amount of time for a generation.
data['age_bin'] = pd.cut(data['age'], bins=[0,28,38,49,65,93], labels=['(17-28]', '(28-38]', '(38-49]', '(49-65]', '(65-93]'])

In [24]:
data['age_bin'].value_counts().sort_index() # check the frequency of each bin

#### 'workinghours' column: extra analysis and binning

In [25]:
# plot of the distribution of the 'workinghours' column
p = sns.displot(data['workinghours'], kde=True, bins=20)
p.set(xlabel='Workinghours', ylabel='Count', title='Distribution of workinghours')
plt.show()

In [26]:
# we check for bins of size 5, the frequency of each bin
pd.cut(data['workinghours'], bins=range(0, 105, 5)).value_counts().sort_index()

In [27]:
data['workinghours'][(data['workinghours'] >= 40) & (data['workinghours'] <= 45)].value_counts()

In [28]:
# we will do customized binning for the 'workinghours' column; 'Part-time' (0-30 hours), 'Full-time' (31-40 hours), and 'Overtime' (41-99 hours)
# https://www.bls.gov/cps/definitions.htm#fullparttime
# https://www.glassdoor.com/blog/guide/how-many-hours-is-part-time/
data['workinghours_bin'] = pd.cut(data['workinghours'], bins=[0, 30, 40, 99], labels=['Part-time', 'Full-time', 'Overtime'])

In [29]:
data['workinghours_bin'].value_counts().sort_index() # check the frequency of each bin

#### 'education' column: extra analysis and binning

In [30]:
# plot of the distribution of the 'education' column
p = sns.displot(data['education'], kde=True, bins=20)
p.set(xlabel='Education', ylabel='Count', title='Distribution of education')
plt.show()

In [31]:
# percentage of people that have an education of atleast 16
data['education'][data['education'] >= 16].value_counts().sum() / len(data) * 100

We see on the graph, and in the percentage, that most people (~90%) have a somewhat higher education level.

In [32]:
data['education'].value_counts().sort_index() # check the frequency of each education level

In [33]:
# we will do customized binning for the 'education' column; https://ilostat.ilo.org/resources/concepts-and-definitions/classification-education/
data['education_bin'] = pd.cut(data['education'], bins=[0, 3, 12, 20, 24], labels=['Less than basic', 'Basic', 'Intermediate', 'Advanced'])

In [34]:
data['education_bin'].value_counts().sort_index() / len(data) # check the frequency of each bin

### Extra inspection of the dataset

In [35]:
# we check how many did not give birth this year in percentage
data['gave birth this year'].value_counts() / len(data) * 100

In [36]:
counts = data.groupby(['sex', 'income']).size().unstack(fill_value=0)
total_counts = data['sex'].value_counts()
# Calculate the percentage of each income level for each sex
percentage = counts.div(total_counts, axis=0) * 100

In [37]:
percentage

In [38]:
# check value count of high and low income
data['income'].value_counts() / len(data) * 100

Above, we can see the income distribution ('high' or 'low') in percentages for male and female, and also the distribution of high and low income in general.

In [39]:
# we check the amount of men that work in the 'private' sector
male_private_count = data[(data['sex'] == 'Male') & (data['workclass'] == 'private')].shape[0]

In [40]:
male_private_count

In [41]:
male_private_count / len(data) * 100

In [42]:
# check amount of rows with wife, low and female
data[(data['marital status'] == 'Wife') & (data['income'] == 'low') & (data['sex'] == 'Female')].shape[0] / len(data)

In [43]:
data['occupation'].value_counts() / len(data) * 100

### Final step

In [44]:
data = data.drop(columns=['age', 'workinghours', 'education']) # we drop the original columns

In [45]:
data

### Writing the cleaned dataset to a new file

In [47]:
# write cleaned dataset to new file
save_dataset(data, "../data/assignment1_income_levels_cleaned.xlsx")